In [1]:
# Item Based Collaborative Filtering

import sys
import random
import math
import os
from operator import itemgetter
from collections import defaultdict

# 使得随机数据可预测，即只要seed的值一样，后续生成的随机数都一样
random.seed(0)


class ItemBasedCF(object):
    ''' TopN recommendation - Item Based Collaborative Filtering '''

    def __init__(self):
        self.trainset = {}
        self.testset = {}

        # n_sim_movie: top 20个电影， n_rec_movie: top 10个推荐结果
        self.n_sim_movie = 20
        self.n_rec_movie = 10

        # movie_sim_mat: 电影之间的相似度， movie_popular: 电影的出现次数， movie_count: 总电影数量
        self.movie_sim_mat = {}
        self.movie_popular = {}
        self.movie_count = 0

        print('Similar movie number = %d' % self.n_sim_movie, file=sys.stderr)  # sys.stderr 目的就是返回错误信息
        print('Recommended movie number = %d' % self.n_rec_movie, file=sys.stderr)

    @staticmethod
    def loadfile(filename):
        ''' load a file, return a generator. '''
        fp = open(filename, 'r')
        # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
        for i, line in enumerate(fp):
            # yield和return的关系和区别：
            # return在程序中返回某个值，返回之后程序就不再往下运行了。
            # 带yield的函数是一个生成器，而不是一个函数，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，
            # 这一次的next开始的地方是接着上一次的next停止的地方执行的，然后遇到yield后，return出要生成的数，此步就结束。
            yield line.strip('\r\n')
            if i % 100000 == 0:
                print ('loading %s(%s)' % (filename, i), file=sys.stderr)
        fp.close()
        print ('load %s succ' % filename, file=sys.stderr)

    def generate_dataset(self, filename, pivot=0.7):
        ''' load rating data and split it to training set and test set '''
        trainset_len = 0
        testset_len = 0

        for line in self.loadfile(filename):
            # 用户ID，电影名称，评分，时间戳
            user, movie, rating, _ = line.split('::')
            # split the data by pivot
            if random.random() < pivot:
                # dict.setdefault(key, default=None), 如果键不存在于字典中，将会添加键并将值设为默认值
                self.trainset.setdefault(user, {})
                self.trainset[user][movie] = int(rating)
                trainset_len += 1
            else:
                self.testset.setdefault(user, {})
                self.testset[user][movie] = int(rating)
                testset_len += 1

        print ('split training set and test set succ', file=sys.stderr)
        print ('train set = %s' % trainset_len, file=sys.stderr)
        print ('test set = %s' % testset_len, file=sys.stderr)

    def calc_movie_sim(self):
        ''' calculate movie similarity matrix '''
        print('counting movies number and popularity...', file=sys.stderr)

        for user, movies in self.trainset.items():  # dict.items() 以列表返回可遍历的(键, 值) 元组数组
            for movie in movies:
                # count item popularity
                if movie not in self.movie_popular:
                    self.movie_popular[movie] = 0
                self.movie_popular[movie] += 1

        print('count movies number and popularity succ', file=sys.stderr)

        # save the total number of movies
        self.movie_count = len(self.movie_popular)
        print('total movie number = %d' % self.movie_count, file=sys.stderr)

        # count co-rated users between items
        itemsim_mat = self.movie_sim_mat
        print('building co-rated users matrix...', file=sys.stderr)

        for user, movies in self.trainset.items():
            for m1 in movies:
                itemsim_mat.setdefault(m1, defaultdict(int))
                for m2 in movies:
                    if m1 == m2:
                        continue
                    itemsim_mat[m1][m2] += 1

        print('build co-rated users matrix succ', file=sys.stderr)

        # calculate similarity matrix
        print('calculating movie similarity matrix...', file=sys.stderr)
        simfactor_count = 0
        PRINT_STEP = 2000000

        for m1, related_movies in itemsim_mat.items():
            for m2, count in related_movies.items():
                # 余弦相似度
                itemsim_mat[m1][m2] = count / math.sqrt(
                    self.movie_popular[m1] * self.movie_popular[m2])
                simfactor_count += 1
                # 打印进度条
                if simfactor_count % PRINT_STEP == 0:
                    print('calculating movie similarity factor(%d)' %
                          simfactor_count, file=sys.stderr)

        print('calculate movie similarity matrix(similarity factor) succ',
              file=sys.stderr)
        print('Total similarity factor number = %d' %
              simfactor_count, file=sys.stderr)

    def recommend(self, user):
        ''' Find K similar movies and recommend N movies '''
        K = self.n_sim_movie
        N = self.n_rec_movie
        rank = {}
        watched_movies = self.trainset[user]

        for movie, rating in watched_movies.items():
            # itemgetter函数用于获取对象的哪些维的数据
            for related_movie, similarity_factor in sorted(self.movie_sim_mat[movie].items(),
                                                           key=itemgetter(1), reverse=True)[:K]:
                if related_movie in watched_movies:
                    continue
                rank.setdefault(related_movie, 0)
                rank[related_movie] += similarity_factor * rating
                
        # return the N best movies
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[:N]

    def evaluate(self):
        ''' print evaluation result: precision, recall, coverage and popularity '''
        print('Evaluation start...', file=sys.stderr)

        N = self.n_rec_movie
        #  varables for precision and recall
        hit = 0  # hit 命中(测试集和推荐集相同+1)
        rec_count = 0  # rec_count 每个用户的推荐数
        test_count = 0  # test_count 每个用户对应的测试数据集的电影数
        # varables for coverage
        all_rec_movies = set()
        # varables for popularity
        popular_sum = 0

        # 对比测试集和推荐集的差异
        for i, user in enumerate(self.trainset):
            if i % 500 == 0:
                print ('recommended for %d users' % i, file=sys.stderr)
            test_movies = self.testset.get(user, {})
            rec_movies = self.recommend(user)
            for movie, _ in rec_movies:
                if movie in test_movies:
                    hit += 1
                all_rec_movies.add(movie)
                popular_sum += math.log(1 + self.movie_popular[movie])
            rec_count += N
            test_count += len(test_movies)

        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        coverage = len(all_rec_movies) / (1.0 * self.movie_count)
        popularity = popular_sum / (1.0 * rec_count)

        print ('precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f' %
               (precision, recall, coverage, popularity), file=sys.stderr)
        

if __name__ == '__main__':

    # 创建ItemCF对象
    itemcf = ItemBasedCF()
    
    # 将数据按照 7:3的比例，拆分成：训练集和测试集，存储在itemcf的trainset和testset中
    ratingfile = os.path.join('ml-1m', 'ratings.dat')
    itemcf.generate_dataset(ratingfile)

    # 计算电影之间的相似度
    itemcf.calc_movie_sim()
    
    # 评估推荐效果
    itemcf.evaluate()

Similar movie number = 20
Recommended movie number = 10
loading ml-1m\ratings.dat(0)
loading ml-1m\ratings.dat(100000)
loading ml-1m\ratings.dat(200000)
loading ml-1m\ratings.dat(300000)
loading ml-1m\ratings.dat(400000)
loading ml-1m\ratings.dat(500000)
loading ml-1m\ratings.dat(600000)
loading ml-1m\ratings.dat(700000)
loading ml-1m\ratings.dat(800000)
loading ml-1m\ratings.dat(900000)
loading ml-1m\ratings.dat(1000000)
load ml-1m\ratings.dat succ
split training set and test set succ
train set = 700450
test set = 299759
counting movies number and popularity...
count movies number and popularity succ
total movie number = 3666
building co-rated users matrix...
build co-rated users matrix succ
calculating movie similarity matrix...
calculating movie similarity factor(2000000)
calculating movie similarity factor(4000000)
calculating movie similarity factor(6000000)
calculating movie similarity factor(8000000)
calculating movie similarity factor(10000000)
calculate movie similarity matrix

In [2]:
# 查看某用户推荐结果
user = "2"
print("推荐结果", itemcf.recommend(user))
print("---", itemcf.testset.get(user, {}))

推荐结果 [('1198', 29.13938797234213), ('2916', 28.560429600116162), ('1580', 28.343757632284298), ('648', 27.590704242397226), ('377', 27.16447361911193), ('260', 26.738771808640664), ('1610', 26.703874275495536), ('608', 26.145701005064332), ('733', 24.690396954573032), ('1036', 18.90862626768337)]
--- {'1537': 4, '648': 4, '2628': 3, '2916': 3, '3108': 3, '3035': 4, '1253': 3, '1610': 5, '292': 3, '2236': 5, '368': 4, '3147': 5, '1544': 4, '1293': 5, '3255': 4, '2278': 3, '2490': 3, '1834': 4, '2852': 3, '982': 4, '1225': 5, '515': 5, '3699': 2, '2353': 4, '590': 5, '1198': 4, '593': 5, '1955': 4, '1957': 5, '163': 4, '21': 1, '1090': 2, '349': 4, '2728': 3, '2943': 4, '3678': 3, '1246': 5}
